In [9]:
!pip install nba_api

     ------------------------------------ 255.2/255.2 kB 682.3 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# ライブデータ

In [14]:
from nba_api.live.nba.endpoints import scoreboard

# Today's Score Board
games = scoreboard.ScoreBoard()

# json
games.get_json()

# dictionary
games.get_dict()

{'meta': {'version': 1,
  'request': 'https://nba-prod-us-east-1-mediaops-stats.s3.amazonaws.com/NBA/liveData/scoreboard/todaysScoreboard_00.json',
  'time': '2023-03-20 04:06:20.620',
  'code': 200},
 'scoreboard': {'gameDate': '2023-03-19',
  'leagueId': '00',
  'leagueName': 'National Basketball Association',
  'games': [{'gameId': '0022201063',
    'gameCode': '20230319/DENBKN',
    'gameStatus': 3,
    'gameStatusText': 'Final',
    'period': 4,
    'gameClock': '',
    'gameTimeUTC': '2023-03-19T19:30:00Z',
    'gameEt': '2023-03-19T15:30:00Z',
    'regulationPeriods': 4,
    'ifNecessary': False,
    'seriesGameNumber': '',
    'seriesText': '',
    'homeTeam': {'teamId': 1610612751,
     'teamName': 'Nets',
     'teamCity': 'Brooklyn',
     'teamTricode': 'BKN',
     'wins': 39,
     'losses': 32,
     'score': 102,
     'seed': None,
     'inBonus': None,
     'timeoutsRemaining': 0,
     'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 21},
      {'period': 2, 'per

## 特定のチームの直近三試合ゲームID取得(test)

In [38]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog

# チーム名を指定
team_name = 'Los Angeles Lakers'

# チームIDを取得
team_id = teams.find_teams_by_full_name(team_name)[0]['id']

# 直近過去3試合の試合IDを取得
gamelog = teamgamelog.TeamGameLog(team_id=team_id).get_data_frames()[0]
game_ids = gamelog['Game_ID'].head(3).tolist()

print(game_ids)

['0022201070', '0022201054', '0022201038']


# ゲームIDからBoxScoreSummaryV2を使って試合データ取得。必要な情報を抜き出す

In [120]:
# まとめ
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import BoxScoreSummaryV2

# チーム名を指定
team_name = 'Los Angeles Lakers'

# チームIDを取得
team_id = teams.find_teams_by_full_name(team_name)[0]['id']

# 直近過去3試合の試合IDを取得
gamelog = teamgamelog.TeamGameLog(team_id=team_id).get_data_frames()[0]
game_ids = gamelog['Game_ID'].head(3).tolist()

# 試合IDを指定
for game_id in game_ids:

    # BoxScoreSummaryV2を使って試合データを取得
    boxscore_summary = BoxScoreSummaryV2(game_id=game_id).get_data_frames()

    # チーム名を取得
    team_info = boxscore_summary[1]
    team_name_1 = team_info['TEAM_ABBREVIATION'].iloc[0]
    team_name_2 = team_info['TEAM_ABBREVIATION'].iloc[1]

    # 総合得点と各クォーターの得点を取得する
    team_stats = boxscore_summary[5]
    team_score_1 = team_stats['PTS'].iloc[0]
    team_score_2 = team_stats['PTS'].iloc[1]
    
    q1_score_1 = team_stats['PTS_QTR1'].iloc[0]
    q2_score_1 = team_stats['PTS_QTR2'].iloc[0]
    q3_score_1 = team_stats['PTS_QTR3'].iloc[0]
    q4_score_1 = team_stats['PTS_QTR4'].iloc[0]
    q1_score_2 = team_stats['PTS_QTR1'].iloc[1]
    q2_score_2 = team_stats['PTS_QTR2'].iloc[1]
    q3_score_2 = team_stats['PTS_QTR3'].iloc[1]
    q4_score_2 = team_stats['PTS_QTR4'].iloc[1]

    # オーバータイムのスコアを取得
    if team_stats['PTS_OT1'].iloc[0] == False:
        print(f'{team_name_1} {team_score_1} - {team_score_2} {team_name_2}')
    else:
        ot_scores = []
        for i in team_stats:
            ot_score_1 = team_stats[f'PTS_OT{i+1}'].iloc[0]
            ot_score_2 = team_stats[f'PTS_OT{i+1}'].iloc[1]
            ot_scores.append(f'OT{i+1}: {ot_score_1}-{ot_score_2}')
        print(f'{team_name_1} {team_score_1} ({", ".join(ot_scores)}) - {team_score_2} {team_name_2}')
    print(f'Q1: {q1_score_1}-{q1_score_2}, Q2: {q2_score_1}-{q2_score_2}, Q3: {q3_score_1}-{q3_score_2}, Q4: {q4_score_1}-{q4_score_2}')

ORL 111 - 105 LAL
Q1: 28-25, Q2: 31-24, Q3: 25-32, Q4: 27-24
LAL 111 - 110 DAL
Q1: 29-33, Q2: 33-21, Q3: 27-26, Q4: 22-30
LAL 110 - 114 HOU
Q1: 19-31, Q2: 29-31, Q3: 26-21, Q4: 36-31
